In [1]:
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import json
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'

In [10]:
def get_columns_and_types(df):
    print(f'Types detected: {", ".join([str(i) for i in df.dtypes.unique()])}')
    return {
        'int': [label for label, dtype in df.dtypes.items() if dtype in [int, np.int64]],
        'float': [label for label, dtype in df.dtypes.items() if dtype in [float, np.float64]],
        'object': [label for label, dtype in df.dtypes.items() if dtype in ['O', 'object']],
    }


def convert_dollars_to_float(s, pattern=r"\$|,"):
    """
    convert money-like strings to floats
    """
    if type(s) is str and re.match(pattern, s):
        # if the input is a str containing '$' and/or ',' try to remove those chars and covert the result to a float
        # if this fails, then there is likely text mixed in (like "$195 this week only!")
        try:
            return float(re.sub(pattern, repl='', string=s))
        except:
            return s
    else:
        # otherwise just return the input
        return s


def estimate_y_from_X(data, y_label, X_labels, train_size=0.6, random_state=42, add_constant=True):
    if type(X_labels) not in [list, tuple]:
        X_labels = [X_labels] # in case we pass a scalar
        
    # we know that we have data for 'price' in all observations, but not for 'monthly_price' or 'weekly_price'
    # drop the nans, create a train/test split, build a model and estimate the monthly, then use the model to fill the missing values in the dataset
    reg_data = data[X_labels + [y_label]].dropna()
    y = reg_data[y_label]
    X = reg_data[X_labels]
    if add_constant:
        X = sm.add_constant(reg_data[X_labels])
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)

    print(f'Estimating {y_label}~1+{"+".join(X_labels)}')
    m = sm.OLS(exog=X_train, endog=y_train).fit()
    
    print(f'... Train fit: {r2_score(y_true=y_train, y_pred=m.predict(exog=X_train)):.2f}')
    print(f'... Test fit: {r2_score(y_true=y_test, y_pred=m.predict(exog=X_test)):.2f}')
    
    print(f'... Filling NaNs in {y_label} with estimated data')
    missing = data[y_label][data[y_label].isnull()].index
    data.loc[missing, y_label] = m.predict(exog=sm.add_constant(data[data[y_label].isnull()][X_labels]))
    return data


def classify_zipcodes_based_on_latitude_and_longitude(data, y_label, X_labels, train_size=0.6, random_state=42):
    # loop over existing zip codes and isolate those that are 5 digit strings: we'll consider these to be accurate
    # otherwise, consider them innacurate and replace them with NaN
    # store this as zipcode_clean, for use in the classification algo
    if 'zipcode_cleaned' in data:
        data = data.drop('zipcode_cleaned', axis=1)

    new_zipcodes = []
    for k, v in data['zipcode'].items():
        if pd.isnull(v) or type(v) is str and len(v) != 5:
            new_zipcodes.append(np.nan)
        else:
            new_zipcodes.append(str(v))
    data = pd.concat([
        data,
        pd.Series(index=data.index, data=new_zipcodes).to_frame('zipcode_cleaned').astype('Int64'),
    ], axis=1)

    if type(X_labels) is str:
        X_labels = [X_labels]
    
    knn_data = data[[y_label] + X_labels].dropna()
    y = knn_data[y_label]
    X = knn_data[X_labels]    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)
    
    print(f'Estimating {y_label} labels from {"+".join(X_labels)}')
    m_knn = KNeighborsClassifier()
    m_knn.fit(X_train, y_train)
    
    print(f'... Train fit: {m_knn.score(X_train, y_train):.2f}')
    print(f'... Test fit: {m_knn.score(X_test, y_test):.2f}')
    
    print(f'... Filling NaNs in {y_label} with estimated data')
    missing = data[y_label][data[y_label].isnull()].index
    data.loc[missing, y_label] = m_knn.predict(data[data[y_label].isnull()][X_labels])
    return data

# Boston Airbnb Dataset
https://www.kaggle.com/datasets/airbnb/boston?resource=download

Possible questions:
- Are there seasonal pattern to the month (or time of time)?
- Is there a seasonal pattern to <time> and location?
- Can we fit a penalized regression model to predict price given <X>?
- Can we show an animated map of Boston, showing average price per zip over time?

In [15]:
boston_listings = pd.read_csv('./data/airbnb_boston/listings.csv', index_col='id')
boston_calendar = pd.read_csv('./data/airbnb_boston/calendar.csv')
boston_reviews = pd.read_csv('./data/airbnb_boston/reviews.csv')

boston_listings = boston_listings.map(convert_dollars_to_float)
boston_calendar = boston_calendar.apply(convert_dollars_to_float)
boston_reviews = boston_reviews.apply(convert_dollars_to_float)

# estimate missing monthly_price/weekly_price fields using regression (based on price field)
boston_listings = estimate_y_from_X(data=boston_listings, y_label='monthly_price', X_labels='price')
boston_listings = estimate_y_from_X(data=boston_listings, y_label='weekly_price', X_labels='price')

# estimate missing zip codes using KNN classification (based on latitude and longitude)
boston_listings = classify_zipcodes_based_on_latitude_and_longitude(data=boston_listings, y_label='zipcode_cleaned', X_labels=['latitude', 'longitude'])

Estimating monthly_price~1+price
... Train fit: 0.75
... Test fit: 0.70
... Filling NaNs in monthly_price with estimated data
Estimating weekly_price~1+price
... Train fit: 0.86
... Test fit: 0.82
... Filling NaNs in weekly_price with estimated data
Estimating zipcode_cleaned labels from latitude+longitude
... Train fit: 0.93
... Test fit: 0.90
... Filling NaNs in zipcode_cleaned with estimated data


In [16]:
# identify column types
boston_listing_col_types = get_columns_and_types(boston_listings)
boston_calendar_col_types = get_columns_and_types(boston_calendar)
boston_review_col_types = get_columns_and_types(boston_reviews)
# boston_listing_col_types

Types detected: object, int64, float64, Int64
Types detected: int64, object
Types detected: int64, object


# clean up zip code info in the listings
- We have missing zip codes (5 digits) but we can estimate it with latitude/longitude
- Regression doesn't make sense here (zip codes are labels, not a function of lat/lon)
- We can use a classifier to do the estimation

# Add a choropleth (geographical map) and overlay the listings prices from the dataset.
- Thank you for the geojson data: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/boston.geojson?short_path=46589b4
- Use log scale for the prices (so the colorbar isn't too compressed) - or change the scale of the colorbar.
- 

In [17]:
# geoplot of neighborhoods with listings overlaid
with open('./data/airbnb_boston/boston.geojson', 'r') as f:
    geojson = json.load(f)
    
df = {'name': [i['properties']['name'] for i in geojson['features']]}
plot_data = boston_listings[['latitude', 'longitude', 'zipcode', 'price']].dropna()
plot_data['log_price'] = plot_data['price'].apply(np.log)

fig = px.choropleth(
    data_frame=df, 
    geojson=geojson, 
    locations='name', 
    featureidkey="properties.name",
    title='Boston Neighborhoods and Airbnb Prices<br>Colorbar is Log Scale'
)
fig.update_geos(fitbounds="locations", visible=False) 

fig.add_trace(
    px.scatter_geo(
        data_frame=plot_data, 
        lat='latitude', 
        lon='longitude', 
        color='log_price',
        hover_data={
            'latitude': ':.2f', 
            'longitude': ':.2f', 
            'price': ':.2f', 
            'log_price': ':.2f',
        },
    ).data[0])

# relabel the colorbar (as showing log values is confusing)
fig.update_coloraxes(colorbar={
    'title': 'Price',
    'tickvals': plot_data['log_price'].quantile([0.01, 0.999]).values,
    'ticktext': ['Cheaper', 'Pricier'],
})
fig.update_layout(showlegend=False)

fig.show()

In [19]:
boston_listings[['price', 'weekly_price', 'monthly_price', 'zipcode_cleaned']].dropna().groupby('zipcode_cleaned').mean().sort_values('price')

,price,weekly_price,monthly_price
zipcode_cleaned,,,
2169,25.000000,223.503112,553.646843
2186,50.000000,363.571546,1097.043362
2472,60.000000,419.598920,1314.401970
2467,80.400000,533.862677,1757.813529
2126,80.958333,504.960583,1712.222253
2124,81.200000,520.634146,1696.737953
2163,86.666667,569.005250,1894.024924
2136,88.793103,567.529127,1946.420140
2145,89.833333,586.747252,1962.855149


In [ ]:
boston_listings[boston_listing_col_types['int']].sort_index()

In [ ]:
boston_listings['zipcode']

In [ ]:
boston_listings[boston_listing_col_types['float']].sort_index().dropna(how='all', axis=1)

In [ ]:
sns.scatterplot(
    data=boston_listings[['price', 'square_feet']].dropna(),
    x='square_feet', y='price')

In [ ]:
sns.pairplot(
    boston_listings[boston_listing_col_types['float']].sort_index().dropna(how='all', axis=1) \
        .drop(['latitude', 'longitude', 'weekly_price', 'monthly_price'], axis=1)
)

In [ ]:
boston_calendar[boston_calendar.price.isnull()]

In [ ]:
boston_calendar[boston_calendar.price.notnull()]

In [ ]:
# convert price column to a float, assuming format of $#.#
boston_calendar.price = [float(i.replace('$', '').replace(',', '')) if type(i) is str else i for i in boston_calendar.price]

# concert date to a datetime object
boston_calendar.date = [pd.Timestamp(i) if type(i) is str else i for i in boston_calendar.date]

# add a month, year columns for seasonality analysis
boston_calendar['month'] = [i.month for i in boston_calendar.date]
boston_calendar['year'] = [i.year for i in boston_calendar.date]

In [ ]:
sns.boxplot(
    data=boston_calendar.dropna(),
    x='month',
    y='price',
)

# Seattle Airbnb Dataset
https://www.kaggle.com/datasets/airbnb/seattle/data

In [ ]:
seattle_listings = pd.read_csv('./data/airbnb_seattle/listings.csv', index_col='id')

In [ ]:
seattle_col_types = get_columns_and_types(seattle_listings)

# Stackoverflow Survey Dataset
https://www.kaggle.com/datasets/stackoverflow/so-survey-2017